In [ ]:
import tvm

import numpy as np

import tvm.te as te
import tvm.relax as rx
import tvm.tir as tir

from tvm.script import ir as I
from tvm.script import tir as T
from tvm.script import relax as R


def showmod(mod: tvm.ir.module.IRModule):
    mod.show(
        black_format=True,
        show_meta=False,
        verbose_expr=True,
        show_object_address=False,
        show_all_struct_info=True,
    )


def createandshowmod(ops):
    te_func = te.create_prim_func(ops).with_attrs({"global_symbol": "test"})
    mod = tvm.IRModule({"test": te_func})
    showmod(mod)


from tvm.relax.frontend import nn

In [ ]:
# exporter.py is the helper file used by function `export_tvm`
def test_builtin_module():
    mod = nn.modules.ReLU()
    export_mod, _ = mod.export_tvm(
        {"forward": {"x": nn.spec.Tensor((4, 4), "float32")}}
    )
    showmod(export_mod)


test_builtin_module()

In [ ]:
def test_custom_module():
    class MyModule(nn.Module):
        def __init__(self, in_features, out_features):
            super().__init__()
            self.in_features = in_features
            self.out_features = out_features
            self.linear_relu_stack = nn.ModuleList(
                [
                    nn.Linear(self.in_features, 512, bias=False),
                    nn.ReLU(),
                    nn.Linear(512, self.out_features, bias=False),
                    nn.ReLU(),
                ]
            )

        def forward(self, x: nn.Tensor):
            return self.linear_relu_stack(x)

    mod = MyModule(10, 10)
    tvm_mod, param_spec = mod.export_tvm(
        {"forward": {"x": nn.spec.Tensor((1, 10), "float32")}}
    )
    showmod(tvm_mod)
    print(param_spec)

    ex = rx.build(tvm_mod, tvm.target.Target("llvm", "llvm"))
    vm = rx.VirtualMachine(ex, tvm.cpu())

    in_data = tvm.nd.array(np.random.rand(1, 10).astype("float32"))
    params = [np.random.rand(*param.shape).astype("float32") for _, param in param_spec]
    params = [tvm.nd.array(param) for param in params]

    print(vm["forward"](in_data, *params).numpy())


test_custom_module()

[('linear_relu_stack.0.weight', Tensor([512, 10], "float32")), ('linear_relu_stack.2.weight', Tensor([10, 512], "float32"))]
[[684.99567 691.64685 744.79456 681.75397 727.8239  712.7559  703.6064
  730.2334  715.39984 704.79254]]


In [ ]:
def test_dynamic_shape():
    linear = nn.Linear(128, 128, bias=False)
    exported_mod, param_spec = linear.export_tvm(
        spec={"forward": {"x": nn.spec.Tensor(("batch_size", 128), "float32")}}
    )
    exported_mod.show()


test_dynamic_shape()